# HealthGPT Colab Quickstart Notebook

This notebook walks through installing dependencies, running a quick inference demo, and training/inferencing the comprehension, NER, and generation models on vlhealth-style data.

Sections:
1. Setup: install libs & clone repo
2. Quick verification (small models)
3. Train comprehension classifier (example)
4. Train NER model (example)
5. Train generation model (example)
6. Inference using trained checkpoints
7. Optional: run FastAPI + ngrok

Notes: Use a GPU runtime for training. Adjust batch sizes / epochs for Colab memory constraints.

In [ ]:
# 1) Install dependencies and clone the repo
!pip install -q transformers datasets evaluate torch accelerate fastapi uvicorn pyngrok nest_asyncio
!rm -rf HealthGPT
!git clone https://github.com/SathwikGudidevuni/HealthGPT.git

## 2) Quick verification (recommended)
This runs a lightweight inference test using small/distilled models so the notebook is quick and fits Colab memory.
It does NOT require training.

In [ ]:
# Use smaller models to avoid OOM
import os, sys
sys.path.append('/content/HealthGPT')
os.environ['COMPREHENSION_ZS_MODEL'] = 'valhalla/distilbart-mnli-12-1'
os.environ['GENERATION_MODEL'] = 't5-small'

from app.pipeline import HealthPipeline
print('Initializing pipeline (may download models)...')
pipe = HealthPipeline()

examples = [
    "I've been feeling dizzy and have a mild fever for two days.",
    "My throat hurts and I have a headache since yesterday.",
    "What medicine should I take for my headache?"
]
from pprint import pprint
for text in examples:
    print('\n=== USER INPUT ===')
    print(text)
    res = pipe.run(text)
    print('\n-- CONTEXT --')
    pprint(res['context'])
    print('\n-- RESPONSE --')
    print(res['response'])

## 3) Train comprehension classifier (example)
This runs the train_comprehension.py script. For quick experiments, use the sample CSVs in data/. Increase epochs/adjust lr_grid for real vlhealth data.

In [ ]:
# Example training command (adjust paths for your data)
!python /content/HealthGPT/train/train_comprehension.py \
  --train_csv /content/HealthGPT/data/sample_comp_train.csv \
  --val_csv /content/HealthGPT/data/sample_comp_train.csv \
  --output_dir /content/outputs/comp_classifier \
  --model_name bert-base-uncased \
  --num_train_epochs 1 \
  --lr_grid 5e-5,3e-5,2e-5 \
  --per_device_train_batch_size 4

## 4) Train NER (token-classification) example
Prepare NER JSON or CSV with tokens and ner_tags as described in train/train_ner.py.

In [ ]:
# Example NER training command (requires proper dataset)
# !python /content/HealthGPT/train/train_ner.py \
#   --train_json /content/HealthGPT/data/ner_train.json \
#   --val_json /content/HealthGPT/data/ner_val.json \
#   --output_dir /content/outputs/ner_model \
#   --model_name dslim/bert-base-NER \
#   --num_train_epochs 3 
print('Skip NER training by default; add your dataset and uncomment the command above')

## 5) Train generation model (example)
Train the seq2seq generator (T5/Flan-T5) on input_text -> target_text. Use small models for Colab.

In [ ]:
# Example generation training (adjust to your dataset)
!python /content/HealthGPT/train/train_generation.py \
  --train_csv /content/HealthGPT/data/sample_gen_train.csv \
  --val_csv /content/HealthGPT/data/sample_gen_train.csv \
  --output_dir /content/outputs/gen_t5 \
  --model_name t5-small \
  --num_train_epochs 1 \
  --per_device_train_batch_size 2

## 6) Inference with trained checkpoints
After training, set the COMP_CLASSIFIER_PATH, COMP_NER_PATH and GENERATION_MODEL env vars and run the pipeline.

In [ ]:
# Example: point to trained outputs and run inference
import os, sys
sys.path.append('/content/HealthGPT')
# adjust these paths based on your actual training outputs
os.environ['COMP_CLASSIFIER_PATH'] = '/content/outputs/comp_classifier/lr_3e-05'
os.environ['COMP_NER_PATH'] = '/content/outputs/ner_model'
os.environ['GENERATION_MODEL'] = '/content/outputs/gen_t5'
from app.pipeline import HealthPipeline
pipe = HealthPipeline()
res = pipe.run("I have a sore throat and a low fever for two days.")
from pprint import pprint
pprint(res)

## 7) Optional: run FastAPI in Colab and expose via ngrok
Only do this if models fit in memory. This starts the API and gives you a public URL.

In [ ]:
# Optional: start FastAPI + ngrok
from pyngrok import ngrok
import nest_asyncio, uvicorn, threading, os, sys
sys.path.append('/content/HealthGPT')
# create public tunnel
public_url = ngrok.connect(8000).public_url
print('ngrok public URL:', public_url)
nest_asyncio.apply()
def run_uvicorn():
    uvicorn.run('app.api:app', host='0.0.0.0', port=8000, log_level='info')
t = threading.Thread(target=run_uvicorn, daemon=True)
t.start()
print('uvicorn started')

## Notes & troubleshooting
- If you see CUDA OOM errors, reduce batch sizes or force CPU by running `os.environ['CUDA_VISIBLE_DEVICES']=''` before importing torch/transformers.
- First-run downloads may take several minutes.
- To persist outputs across Colab sessions, mount Google Drive and copy /content/outputs to Drive.
- For real vlhealth experiments, increase --num_train_epochs, tune lr_grid, and gather more labeled data.